In [ ]:
from entsoe import EntsoePandasClient
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

%matplotlib inline

client = EntsoePandasClient(api_key="6910a80c-6c28-4837-9131-e8caa1bf9de2")

start = pd.Timestamp("2024-06-01", tz="UTC")
end = pd.Timestamp("2024-06-04", tz="UTC")

country_code = 'DE'  # Germany

# Forecasts for wind and solar (onshore + offshore + solar)
forecast = client.query_wind_and_solar_forecast(country_code, start=start, end=end)
forecast = forecast[['Wind Onshore', 'Wind Offshore']]  # Keep only wind components
forecast['Total Forecast'] = forecast.sum(axis=1)
forecast.head()

# Actual generation data (all types)
actual = client.query_generation(country_code, start=start, end=end)

# Keep only wind types
actual = actual[['Wind Onshore', 'Wind Offshore']]
actual['Total Actual'] = actual.sum(axis=1)
actual.head()

df = pd.concat([
    forecast['Total Forecast'].rename('Forecast'),
    actual['Total Actual'].rename('Actual')
], axis=1)

df.dropna(inplace=True)  # In case of mismatches
df['Forecast Error (%)'] = 100 * (df['Forecast'] - df['Actual']) / df['Actual']
df.head()


df[['Forecast', 'Actual']].plot(figsize=(10,5), title="Wind Forecast vs Actual (MW)")
plt.ylabel("MW")
plt.show()

df['Forecast Error (%)'].plot(figsize=(10,4), title="Forecast Error (%)")
plt.axhline(0, color='black', linestyle='--')
plt.ylabel("%")
plt.show()

# Querying day-ahead market prices
da_prices = client.query_day_ahead_prices(country_code, start=start,end=end)
print(da_prices.head())

: 